# Cameras and images

This demo shows with the example of the ``DummyGrabber``
how you acquire an image and apply some filters.


In [ ]:
from itom import dataIO
from itom import dataObject
from itom import algorithms
from itom import liveImage
from itom import saveIDC
from itom import loadIDC
from itom import plot

Initialize a ``DummyGrabber`` camera



In [ ]:
cam = dataIO("DummyGrabber")
cam.setParam("bpp", 8)
# start camera (only once)
cam.startDevice()

# show live image of camera
liveImage(cam)

.. hint:: The live images tries to acquire and get up to 50 images per second
    from the camera. If you want to acquire images by yourself in a script, you need
    to stop the timer of the live images for a certain amount of time. After you are
    done with your manual acquisition, you can restart the timer again.



In [ ]:
currentStatus = cam.getAutoGrabbing()
print("Current value of auto grabbing property of the camera:", currentStatus)

cam.setAutoGrabbing(False)

Acquire 10 images in a list of dataObjects



In [ ]:
result = []
d = dataObject()  # empty data object where the image should be put in

for i in range(0, 10):
    cam.acquire()
    cam.getVal(d)  # d is a shallow copy of the camera image
    result.append(d.copy())

Save the list of images to the **image1.idc** file (idc is a file format for the python pickle module)



In [ ]:
saveIDC("image1.idc", {"result": result, "description": "sample 1"})

Load the list of images



In [ ]:
loaded_objects = loadIDC("image1.idc")
result2 = loaded_objects["result"]

# plot the 3rd image from the list
plot(result2[2])

Acquire 10 images in an image stack



In [ ]:
num = 10
sizex = cam.getParam("sizex")
sizey = cam.getParam("sizey")
bpp = cam.getParam("bpp")

if bpp == 8:
    d = dataObject([num, sizey, sizex], "uint8")
else:
    d = dataObject([num, sizey, sizex], "uint16")

for idx in range(num):
    cam.acquire()
    cam.copyVal(d[idx, :, :])  # partial deep copy into one part of the 3d object d

plot(d)

Calculate mean value of image stack in z-direction.



In [ ]:
result_mean = dataObject()

algorithms.calcMeanZ(d, result_mean, ignoreInf=0, calcStd=0)
# result_mean is a 3d Object with [1 x sizey x sizex] dimensions.
# We squeeze it to get a 2D Object
result_mean = result_mean.squeeze()

result_mean.setTag("title", f"mean value of {num} acquisitions")
result_mean.axisUnits = ("px", "px")
result_mean.axisDescriptions = ("y", "x")
plot(result_mean)

<img src="file://../../_static/demoCameraAndImages_1.png" width="100%">



Apply Gaussian filter onto the mean image.



In [ ]:
result_filter = dataObject()
kernelVal = 9
algorithms.gaussianFilter(
    result_mean, result_filter, kernelx=kernelVal, kernely=kernelVal
)

Copy meta information from source ``dataObject``.



In [ ]:
result_filter.copyMetaInfo(result_mean)
result_filter.setTag("title", f"Gaussian filter with kernel {kernelVal}")
plot(result_filter)

<img src="file://../../_static/demoCameraAndImages_2.png" width="100%">



In [ ]:
# reset the auto grabbing functionality of possibly connected live images
cam.setAutoGrabbing(currentStatus)

# end camera
cam.stopDevice()